In [1]:
# import libraries
import pandas as pd
import mysql.connector

In [2]:
# load the data 
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

In [3]:
#mysql connection 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="9825"
)
cur = conn.cursor()

cur.execute("CREATE DATABASE IF NOT EXISTS food_data;")
cur.execute("USE food_data;")
conn.commit()

In [4]:
# --- Create Tables if not exists ---
cur.execute('''
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INT,
        Name VARCHAR(255),
        Type VARCHAR(100),
        Address TEXT,
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID INT,
        Name VARCHAR(255),
        Type VARCHAR(100),
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS food_listings (
        Food_ID INT,
        Food_Name VARCHAR(255),
        Quantity INT,
        Expiry_Date DATE,
        Provider_ID INT,
        Provider_Type VARCHAR(100),
        Location VARCHAR(100),
        Food_Type VARCHAR(50),
        Meal_Type VARCHAR(50)
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID INT,
        Food_ID INT,
        Receiver_ID INT,
        Status VARCHAR(50),
        Timestamp DATETIME
    )
''')

conn.commit()

In [5]:
print(food_listings.columns.tolist())
print(food_listings.head())

['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID', 'Provider_Type', 'Location', 'Food_Type', 'Meal_Type']
   Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43   3/17/2025          110     Grocery Store   
1        2      Soup        22   3/24/2025          791     Grocery Store   
2        3    Fruits        46   3/28/2025          478  Catering Service   
3        4    Fruits        15   3/16/2025          930        Restaurant   
4        5      Soup        14   3/19/2025          279        Restaurant   

           Location       Food_Type  Meal_Type  
0  South Kellyville  Non-Vegetarian  Breakfast  
1        West James  Non-Vegetarian     Dinner  
2       Lake Regina           Vegan  Breakfast  
3         Kellytown           Vegan      Lunch  
4        Garciaport           Vegan     Dinner  


In [6]:
print(claims.columns.tolist())
print(claims.head())

['Claim_ID', 'Food_ID', 'Receiver_ID', 'Status', 'Timestamp']
   Claim_ID  Food_ID  Receiver_ID     Status        Timestamp
0         1      164          908    Pending    3/5/2025 5:26
1         2      353          391  Cancelled  3/11/2025 10:24
2         3      626          492  Completed   3/21/2025 0:59
3         4       61          933  Cancelled    3/4/2025 9:08
4         5      345          229    Pending  3/14/2025 15:17


In [7]:
# Convert date columns to datetime and then to string (MySQL format)
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'], errors='coerce').dt.strftime('%Y-%m-%d')
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')


In [8]:
#load the data
# ---- PROVIDERS ----
for index, row in providers.iterrows():
    cur.execute("""
        INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, tuple(row))

# ---- RECEIVERS ----
for index, row in receivers.iterrows():
    cur.execute("""
        INSERT INTO receivers (Receiver_ID, Name, Type, City, Contact)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))

# ---- FOOD LISTINGS ----
for index, row in food_listings.iterrows():
    cur.execute("""
        INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date,
                                   Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

# ---- CLAIMS ----
for index, row in claims.iterrows():
    cur.execute("""
        INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()

In [9]:
# counting rows 
tables = ["providers", "receivers", "food_listings", "claims"]

for t in tables:
    cur.execute(f"SELECT COUNT(*) FROM {t}")
    count = cur.fetchone()[0]
    print(f"{t}: {count} rows")
    

providers: 1000 rows
receivers: 1000 rows
food_listings: 1000 rows
claims: 1000 rows


**SQL QUERRIES**

In [10]:
# 1. How many food providers and receivers are there in each city?
q1 = """
SELECT city,
       SUM(no_of_providers) AS no_of_providers,
       SUM(no_of_receivers) AS no_of_receivers
FROM (
    SELECT city, COUNT(*) AS no_of_providers, 0 AS no_of_receivers
    FROM providers GROUP BY city
    UNION ALL
    SELECT city, 0 AS no_of_providers, COUNT(*) AS no_of_receivers
    FROM receivers GROUP BY city
) t
GROUP BY city;
"""
cur.execute(q1)
df1 = pd.DataFrame(cur.fetchall(), columns=["City", "No_of_Providers", "No_of_Receivers"])

In [11]:
# 2. Which type of food provider contributes the most food?
q2 = """
SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity DESC;
"""
cur.execute(q2)
df2 = pd.DataFrame(cur.fetchall(), columns=["Provider_Type", "Total_Quantity"])

In [14]:
# 3.What is the contact information of food providers in a specific city?
q3 = """
SELECT city, provider_id, name as provider_name, contact
FROM providers
ORDER BY city;
"""
cur.execute(q3)
df3 = pd.DataFrame(cur.fetchall(), columns=["City", "Provider_ID", "Name", "Contact"])

In [15]:
df3

,City,Provider_ID,Name,Contact
0,Adambury,338,Ibarra LLC,6703380260
1,Adamsview,510,Lozano-Miller,001-281-026-8022
2,Adamsville,757,Davis Ltd,(112)122-3591x558
3,Aguirreville,727,Rowe-Chen,8228891240
4,Alexanderchester,962,Galloway-Henderson,001-867-928-0212x3211
...,...,...,...,...
995,Wrightville,955,Williams PLC,(730)287-3289
996,Yatesside,760,Frederick LLC,784.860.0232x137
997,Youngchester,691,"Obrien, King and Robinson",+1-513-264-8229x419
998,Zimmermanton,928,Larson and Sons,152.836.4864


In [17]:
# 4. Which receivers have claimed the most food?
q4 = """
SELECT r.name, COUNT(c.claim_id) AS no_of_claims
FROM claims c
JOIN receivers r 
    ON c.Receiver_ID = r.Receiver_ID
WHERE c.status = 'completed'
GROUP BY c.Receiver_ID, r.name
ORDER BY no_of_claims DESC;
"""
cur.execute(q4)
df4 = pd.DataFrame(cur.fetchall(), columns=["Receiver_Name", "Number_of_Claims"])

In [18]:
# 5. Total quantity of food available
q5 = """
SELECT food_name, SUM(quantity) AS total_quantity
FROM food_listings 
GROUP BY food_name
order by total_quantity desc;
"""
cur.execute(q5)
df5 = pd.DataFrame(cur.fetchall(), columns=["Food_Name", "Total_Quantity"])

In [19]:
# 6. Which city has the highest number of food listings?
q6 = """
SELECT location, COUNT(location) AS no_of_appearance
FROM food_listings
GROUP BY location
order by no_of_appearance desc;
"""
cur.execute(q6)
df6 = pd.DataFrame(cur.fetchall(), columns=["Location", "No_of_Appearance"])

In [20]:
# 7. Most commonly available food types
q7 = """
SELECT Food_Type, COUNT(*) AS Count
FROM food_listings
GROUP BY Food_Type
ORDER BY Count DESC;
"""
cur.execute(q7)
df7 = pd.DataFrame(cur.fetchall(), columns=["Food_Type", "Count"])

In [21]:
# 8. How many food claims have been made for each food item?
q8 = """
SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claims
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.status ='completed'
GROUP BY f.Food_Name
ORDER BY Claims DESC;
"""
cur.execute(q8)
df8 = pd.DataFrame(cur.fetchall(), columns=["Food_Name", "Claims"])


In [22]:
# 9. Provider with highest number of successful claims
q9 = """
SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
JOIN claims c ON f.Food_ID = c.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC;
"""
cur.execute(q9)
df9 = pd.DataFrame(cur.fetchall(), columns=["Provider_Name", "Successful_Claims"])

In [23]:
# 10. % of completed vs pending vs canceled claims
q10 = """
SELECT Status, 
       COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
FROM claims
GROUP BY Status;
"""
cur.execute(q10)
df10 = pd.DataFrame(cur.fetchall(), columns=["Status", "Percentage"])


In [24]:
# 11. Average quantity of food claimed per receiver
q11 = """
SELECT r.Name, round(AVG(f.Quantity)) AS Avg_Quantity
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY r.Name;
"""
cur.execute(q11)
df11 = pd.DataFrame(cur.fetchall(), columns=["Receiver_Name", "Avg_Quantity"])

In [25]:
# 12. Which meal type is claimed the most?
q12 = """
SELECT Meal_Type, COUNT(*) AS Total_Claims
FROM food_listings f
JOIN claims c ON f.Food_ID = c.Food_ID
GROUP BY Meal_Type
ORDER BY Total_Claims DESC;
"""
cur.execute(q12)
df12 = pd.DataFrame(cur.fetchall(), columns=["Meal_Type", "Total_Claims"])


In [26]:
# 13. Total quantity donated by each provider
q13 = """
SELECT p.Name, SUM(f.Quantity) AS Total_Donated
FROM providers p
JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC;
"""
cur.execute(q13)
df13 = pd.DataFrame(cur.fetchall(), columns=["Provider_Name", "Total_Donated"])

In [27]:
# Which city has the highest number of cancelled claims
q14 = """
SELECT r.City, COUNT(c.Claim_ID) AS Cancelled_Claims
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE c.Status = 'cancelled'
GROUP BY r.City
ORDER BY Cancelled_Claims DESC;
"""
cur.execute(q14)
df14 = pd.DataFrame(cur.fetchall(), columns=["City", "Cancelled_Claims"])

In [28]:
# Which food type has the highest number of cancelled claims
q15 = """
SELECT f.Food_Type, COUNT(c.Claim_ID) AS Cancelled_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'cancelled'
GROUP BY f.Food_Type
ORDER BY Cancelled_Claims DESC;
"""
cur.execute(q15)
df15 = pd.DataFrame(cur.fetchall(), columns=["Food_Type", "Cancelled_Claims"])

In [30]:
# Save all DataFrames to CSV files
df1.to_csv("df1.csv", index=False)
df2.to_csv("df2.csv", index=False)
df3.to_csv("df3.csv", index=False)
df4.to_csv("df4.csv", index=False)
df5.to_csv("df5.csv", index=False)
df6.to_csv("df6.csv", index=False)
df7.to_csv("df7.csv", index=False)
df8.to_csv("df8.csv", index=False)
df9.to_csv("df9.csv", index=False)
df10.to_csv("df10.csv", index=False)
df11.to_csv("df11.csv", index=False)
df12.to_csv("df12.csv", index=False)
df13.to_csv("df13.csv", index=False)
df14.to_csv("df14.csv", index=False)
df15.to_csv("df15.csv", index=False)

In [31]:
%%writefile food_management_app.py
import streamlit as st
import pandas as pd

st.title("🍽️ Food Donation Insights Dashboard")

# Load DataFrames
dfs = {}
for i in range(1, 16):
    dfs[f"df{i}"] = pd.read_csv(f"df{i}.csv")

# ----------------- Major Tabs -----------------
tabs = st.tabs([
    "Providers & Receivers",
    "Food Listings",
    "Claims & Distribution",
    "Analysis & Insights"
])

# ----------------- Tab 1: Providers & Receivers -----------------
with tabs[0]:
    st.subheader("Providers & Receivers Insights")

    # Dropdown to select which data to see
    choice = st.selectbox("Select view:", [
        "Providers & Receivers per City",
        "Provider Types Contribution",
        "Provider Contact Information"
    ])

    if choice == "Providers & Receivers per City":
        st.dataframe(dfs["df1"])
        st.bar_chart(dfs["df1"].set_index("City"))
    elif choice == "Provider Types Contribution":
        st.dataframe(dfs["df2"])
        st.bar_chart(dfs["df2"].set_index("Provider_Type"))
    elif choice == "Provider Contact Information":
        selected_city = st.selectbox("Select City", dfs["df3"]["City"].unique())
        st.dataframe(dfs["df3"][dfs["df3"]["City"] == selected_city])

# ----------------- Tab 2: Food Listings -----------------
with tabs[1]:
    st.subheader("Food Listings Insights")
    section = st.radio("Select Section:", [
        "Top Receivers", 
        "Total Food Available", 
        "Number of Food Listings per City/Location",
        "Most Common Food Types"
    ])
    if section == "Top Receivers":
        st.dataframe(dfs["df4"])
        st.bar_chart(dfs["df4"].set_index("Receiver_Name"))
    elif section == "Total Food Available":
        st.dataframe(dfs["df5"])
        st.bar_chart(dfs["df5"].set_index("Food_Name"))
    elif section == "Number of Food Listings per City/Location":
        st.dataframe(dfs["df6"])
        st.bar_chart(dfs["df6"].set_index("Location"))
    elif section == "Most Common Food Types":
        st.dataframe(dfs["df7"])
        st.bar_chart(dfs["df7"].set_index("Food_Type"))

# ----------------- Tab 3: Claims & Distribution -----------------
with tabs[2]:
    st.subheader("Claims & Distribution")
    section = st.selectbox("Select Claim Section:", [
        "Completed Claims per Food Item",
        "Provider with Most Successful Claims",
        "Claims Status Distribution"
    ])
    if section == "Completed Claims per Food Item":
        st.dataframe(dfs["df8"])
        st.bar_chart(dfs["df8"].set_index("Food_Name"))
    elif section == "Provider with Most Successful Claims":
        st.dataframe(dfs["df9"])
    elif section == "Claims Status Distribution":
        st.dataframe(dfs["df10"])
        st.bar_chart(dfs["df10"].set_index("Status"))

# ----------------- Tab 4: Analysis & Insights -----------------
with tabs[3]:
    st.subheader("Analysis & Insights")
    section = st.selectbox("Select Analysis Section:", [
        "Average Quantity per Receiver",
        "Most Claimed Meal Types",
        "Total Donations by Provider",
        "Cancelled Claims by City",
        "Cancelled Claims by Food Type"
    ])
    if section == "Average Quantity per Receiver":
        st.dataframe(dfs["df11"])
    elif section == "Most Claimed Meal Types":
        st.dataframe(dfs["df12"])
        st.bar_chart(dfs["df12"].set_index("Meal_Type"))
    elif section == "Total Donations by Provider":
        st.dataframe(dfs["df13"])
        st.bar_chart(dfs["df13"].set_index("Provider_Name"))
    elif section == "Cancelled Claims by City":
        st.dataframe(dfs["df14"])
        st.bar_chart(dfs["df14"].set_index("City"))
    elif section == "Cancelled Claims by Food Type":
        st.dataframe(dfs["df15"])
        st.bar_chart(dfs["df15"].set_index("Food_Type"))

Overwriting food_management_app.py


In [ ]:
!streamlit run food_management_app.py